In [1]:
!pip install  opencv-contrib-python==4.5.3.56
import numpy as np
import cv2 
import os

You should consider upgrading via the 'c:\users\kittapak\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return (img, img_gray)

In [10]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.78*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 5, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        cv2.imshow('Video frame', detected_img)
        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return

In [4]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [11]:
template_img = cv2.imread('./images/final_exam/Templates/Template-4.png')
template_img, template_gray = preprocessing(template_img)

query_vid = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')
while query_vid.isOpened() :
    ret, frame = query_vid.read()
    
    if ret :
        query_vid_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        detected, drawmatch =  feature_object_detection(template_img, template_gray, frame, query_vid_gray, 10)

        if cv2.waitKey(int(24)) & 0xFF == 27 : # this line control the period between image frame
            break
    else :
        break
query_vid.release()
cv2.destroyAllWindows()
